In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd
import seaborn as sns
from optuna import Study
from datetime import datetime
from src.utils import Run, Sweep, set_directory, INFERENCE_LABELS
from src.analysis.inference import *

# Test errors


In [ ]:
def add_dropout_amount(x: pd.Series):

    if "SGD" not in x.run.cfg.inference._target_ or x.run.cfg.inference.get("use_map", False):
        return x
    else:
        y = x
        y.label = f"SGD (dropout={x.run.cfg.model.dropout})"
        return y

runs = (
    pd.DataFrame(
        [
            {
                "run": Run(p),
                "multirun_start": datetime.strptime(
                    f"{p.parents[1].stem}/{p.parents[0].stem}",
                    r"%Y-%m-%d/%H-%M-%S",
                ),
                "index": int(p.stem),
            }
            for p in (Path("../experiment_results/cifar10_densenet/").glob("*/*/[012]"))
        ]
    )
)

In [ ]:
latest_runs = (
    runs.assign(
        label=lambda x: x.run.map(lambda y: y.inference_label),
        lr=lambda x: x.run.map(
            lambda y: y.cfg.inference.sampler.lr
            if "sampler" in y.cfg.inference
            else y.cfg.inference.lr
        ),
    )
    .apply(add_dropout_amount, axis=1)
    .set_index(["label", "multirun_start", "index"])
    .sort_index()
    .reset_index(level=["label", "index"])
    .drop_duplicates(["label", "index"], keep="last")
    .set_index(["index"], append=True)
    .assign(val_err=lambda x: x.run.map(lambda y: y.get_scalar("err/val").iloc[-1]))
)
latest_runs

In [ ]:
best_runs = latest_runs.loc[lambda x: x.groupby("label").val_err.idxmin()]
best_runs.assign(
    test_err=lambda x: x.run.map(lambda r: r.get_scalar("err/test").item())
)

In [ ]:
all_runs = best_runs.run.tolist()
all_labels =  best_runs.label.tolist()

mcmc_runs = best_runs.loc[lambda x: x.label.str.contains("SGHMC")].run.tolist()
mcmc_labels =  best_runs.loc[lambda x: x.label.str.contains("SGHMC")].label.tolist()


In [ ]:

(
    plot_val_err(all_runs, all_labels)
    .set(ylim=(None, 0.32))
    .savefig("../thesis/Figures/cifar-densenet-final-runs-val.pdf")
)


In [ ]:
test_errs = pd.DataFrame({
    "Method": all_labels,
    "err/test": [x.get_scalar("err/test").item() for x in all_runs]
}).reset_index(drop=True)

In [ ]:
import hydra

dm = hydra.utils.instantiate(best_runs.run[0].cfg.data)
dm.setup()
n_test = len(dm.test_data)
test_errs

In [ ]:
import numpy as np
from math import sqrt


def get_err_incl_ci(error: float) -> str:
    pm = sqrt(error * (1 - error) / n_test) * 1.96
    return f"${100*error:.2f} \\pm {100*pm:.2f}~\\%$"


(
    test_errs
    .apply({"Method": lambda x: x, "err/test": get_err_incl_ci})
    .rename(columns={"err/test": "Test error incl. 95\\% CI"})
    .to_latex("../thesis/Tables/cifar-densenet-test-err.tex", escape=False, index=False)
)

In [ ]:
from textwrap import wrap


# Calibration


In [ ]:
plot_calibration(all_runs, all_labels, legend_cols=4)
plt.savefig("../thesis/Figures/cifar10-densenet-calibration.pdf")

In [ ]:
(
    pd.Series({label: r.get_scalar("ece/test").item() for label, r in zip(best_runs.label, best_runs.run)})
    .rename("ECE")
    .pipe(pd.DataFrame)
    .to_latex("../thesis/Tables/cifar-densenet-ece.tex", formatters={"ECE": lambda x: f"{x:.2%}"})
)


## Checking SGHMC assumptions

In [ ]:
plot_temperatures(mcmc_runs, mcmc_labels)
plt.subplots_adjust(bottom=0.13)
plt.savefig("../thesis/Figures/cifar-densenet-temperatures.pdf")

In [ ]:
from scipy.stats import chi2
from math import sqrt

def binom_95_ci(p: float, n: int):  
    pm = sqrt(p * (1 - p) / n) * 1.96
    return f"${100*p:.2f} \\pm {100*pm:.2f}~\\%$"

def compute_99_chi2(x: pd.Series):
    df = x.index.get_level_values("n_params")[0]
    p = (x < chi2(df).ppf(0.99)).mean()
    n = len(x)
    
    return  binom_95_ci(p, n)

(
    temperatures.set_index("n_params", append=True)
    .groupby(level=["Sampler"])
    .agg(
        frac_in_99=pd.NamedAgg("temperature_sum", aggfunc=compute_99_chi2)
        )
    .rename(columns={"frac_in_99": "$\\hat{T}_{0.99}$"})
    .reset_index()
    .to_latex("../thesis/Tables/cifar-small-temperatures.tex", escape=False, index=False)
)
